In [34]:
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

--2020-04-13 08:19:00--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 52.219.72.208
Connecting to fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)|52.219.72.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26421880 (25M) [binary/octet-stream]
Saving to: ‘train-images-idx3-ubyte.gz.1’

train-images-idx3-u 100%[===================>]  25.20M  11.8MB/s    in 2.1s    

2020-04-13 08:19:02 (11.8 MB/s) - ‘train-images-idx3-ubyte.gz.1’ saved [26421880/26421880]

--2020-04-13 08:19:03--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 52.219.72.208
Connecting to fashion-mnist.s3-website.eu-central-1.

In [35]:
!ls

MNIST_Fashion		     t10k-labels-idx1-ubyte.gz.1
sample_data		     train-images-idx3-ubyte.gz
t10k-images-idx3-ubyte.gz    train-images-idx3-ubyte.gz.1
t10k-images-idx3-ubyte.gz.1  train-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte.gz    train-labels-idx1-ubyte.gz.1


In [0]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import scipy.ndimage.interpolation

In [0]:
#Training PArams
learning_rate = 0.0002
batch_size = 32
epochs = 100000

#Network params
image_dimension = 784 #img sz is 28x28
#Discriminator Nodes
H_dim = 128

def xavier_init(shape):
  return tf.random_normal(shape = shape, stddev= 1./tf.sqrt(shape[0]/2.0))

In [38]:
tf.__version__

'1.2.0'

In [0]:
#define placeholders for external input

X_A = tf.placeholder(tf.float32, shape = [None, image_dimension])
X_B = tf.placeholder(tf.float32, shape = [None, image_dimension])


In [0]:
# Define weights and biases for dictionaries for Discriminator A

Disc_A_W = { "disc_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "disc_final": tf.Variable(xavier_init([H_dim, 1]))}

Disc_A_Bias = { "disc_H" : tf.Variable(xavier_init([H_dim])),
             "disc_final": tf.Variable(xavier_init([1]))}


# Define weights and biases for dictionaries for Discriminator B

Disc_B_W = { "disc_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "disc_final": tf.Variable(xavier_init([H_dim, 1]))}

Disc_B_Bias = { "disc_H" : tf.Variable(xavier_init([H_dim])),
             "disc_final": tf.Variable(xavier_init([1]))}

# Define weights and biases for dictionaries for Generator transforming A to B

Gen_AB_W = { "Gen_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "Gen_final": tf.Variable(xavier_init([H_dim, image_dimension]))} #784 due to output dimension of the image

Gen_AB_Bias = { "Gen_H" : tf.Variable(xavier_init([H_dim])),
             "Gen_final": tf.Variable(xavier_init([image_dimension]))}


# Define weights and biases for dictionaries for Generator transforming B to A

Gen_BA_W = { "Gen_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "Gen_final": tf.Variable(xavier_init([H_dim, image_dimension]))}

Gen_BA_Bias = { "Gen_H" : tf.Variable(xavier_init([H_dim])),
             "Gen_final": tf.Variable(xavier_init([image_dimension]))}


In [0]:
def Disc_A(x):
  D_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Disc_A_W["disc_H"]), Disc_A_Bias["disc_H"]))
  disc_output = (tf.add(tf.matmul(D_hidden_layer, Disc_A_W["disc_final"]), Disc_A_Bias["disc_final"]))
  disc_prob_output = tf.nn.sigmoid(disc_output)
  return disc_prob_output

def Disc_B(x):
  D_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Disc_B_W["disc_H"]), Disc_B_Bias["disc_H"]))
  disc_output = (tf.add(tf.matmul(D_hidden_layer, Disc_B_W["disc_final"]), Disc_B_Bias["disc_final"]))
  disc_prob_output = tf.nn.sigmoid(disc_output)
  return disc_prob_output
  

In [0]:
#Generator NW
def GenAB(x):
  G_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Gen_AB_W["Gen_H"]), Gen_AB_Bias["Gen_H"]))
  Gen_output = (tf.add(tf.matmul(G_hidden_layer, Gen_AB_W["Gen_final"]), Gen_AB_Bias["Gen_final"]))
  Gen_prob_output = tf.nn.sigmoid(Gen_output)
  return Gen_prob_output

def GenBA(x):
  G_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Gen_BA_W["Gen_H"]), Gen_BA_Bias["Gen_H"]))
  Gen_output = (tf.add(tf.matmul(G_hidden_layer, Gen_BA_W["Gen_final"]), Gen_BA_Bias["Gen_final"]))
  Gen_prob_output = tf.nn.sigmoid(Gen_output)
  return Gen_prob_output

In [0]:
# building Cycle GAN NW

# GAN for approximating A Distribution

X_BA = GenBA(X_B)
Disc_A_real = Disc_A(X_A)
Disc_A_fake = Disc_A(X_BA)

# GAN for approximating B Distribution
X_AB = GenBA(X_A)
Disc_B_real = Disc_A(X_B)
Disc_B_fake = Disc_A(X_AB)


In [0]:
# Discriminator Loss Functions

Loss_Disc_A = (tf.reduce_mean(tf.square(Disc_A_real - tf.ones_like(Disc_A_real)) + tf.reduce_mean(tf.square(Disc_A_fake))))/2
Loss_Disc_B = (tf.reduce_mean(tf.square(Disc_B_real - tf.ones_like(Disc_B_real)) + tf.reduce_mean(tf.square(Disc_B_fake))))/2

Disc_Loss = Loss_Disc_A + Loss_Disc_B

In [0]:

# image reconstruction

X_BAB = GenAB(X_BA)
X_ABA = GenBA(X_AB)

In [0]:
 # Generator Loss function
Loss_Gen_A = tf.reduce_mean(tf.square(Disc_B_fake - tf.ones_like(Disc_B_fake)))
Loss_Gen_B = tf.reduce_mean(tf.square(Disc_A_fake - tf.ones_like(Disc_A_fake)))
Loss_total = Loss_Gen_A + Loss_Gen_B

# Reconstruction Loss

Loss_recon_A = tf.reduce_mean(10*tf.abs((X_A-X_ABA)))
Loss_recon_B = tf.reduce_mean(10*tf.abs((X_B-X_BAB)))
Loss_recon_total = Loss_recon_A +Loss_recon_B

Gen_Loss = Loss_total + Loss_recon_total



In [0]:
# parameters list of Discriminator

Disc_param = [Disc_A_W['disc_H'], Disc_A_W["disc_final"], Disc_A_Bias['disc_H'], Disc_A_Bias['disc_final'],
              Disc_B_W['disc_H'], Disc_B_W['disc_final'],Disc_B_Bias['disc_H'], Disc_B_Bias['disc_final'] ]
# parameters list of Gen
Gen_param = [Gen_AB_W['Gen_H'], Gen_AB_W["Gen_final"], Gen_AB_Bias['Gen_H'], Gen_AB_Bias['Gen_final'],
            Gen_BA_W['Gen_H'], Gen_BA_W['Gen_final'],Gen_BA_Bias['Gen_H'], Gen_BA_Bias['Gen_final'] ]


# Define the optimizer

Gen_Optimize = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(Gen_Loss, var_list = Gen_param)
Disc_Optimize = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(Disc_Loss, var_list = Disc_param)



In [49]:
!pip install tensorflow==1.2
!pip install -q git+https://github.com/tensorflow/examples.git
!ls

MNIST_Fashion		     t10k-labels-idx1-ubyte.gz.1
sample_data		     train-images-idx3-ubyte.gz
t10k-images-idx3-ubyte.gz    train-images-idx3-ubyte.gz.1
t10k-images-idx3-ubyte.gz.1  train-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte.gz    train-labels-idx1-ubyte.gz.1


In [0]:
from tensorflow.examples.tutorials.mnist import input_data


In [51]:
!mkdir MNIST_Fashion
!cp *.gz MNIST_Fashion/
# import input_data
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_Fashion/")

# Assembling the training data from 2 domains

X_train = mnist.train.images
mid = int(X_train.shape[0]/2)

# Real Images Dataset 1
X_train_real = X_train[:mid]

# Rotated images Dataset 2
X_train_rot = X_train[mid:].reshape(-1,28,28)
X_train_rot = scipy.ndimage.interpolation.rotate(X_train_rot, 90, axes = (1,2))
X_train_rot = X_train_rot.reshape(-1,28*28)

# random shuffling

def shuffle_data(x, size):
  start_index = np.random.randint(0, x.shape[0]-size)
  return x[start_index:start_index+size]

mkdir: cannot create directory ‘MNIST_Fashion’: File exists
Extracting MNIST_Fashion/train-images-idx3-ubyte.gz
Extracting MNIST_Fashion/train-labels-idx1-ubyte.gz
Extracting MNIST_Fashion/t10k-images-idx3-ubyte.gz
Extracting MNIST_Fashion/t10k-labels-idx1-ubyte.gz


In [52]:
# Initialize the variables

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  X_A_batch = shuffle_data(X_train_real, batch_size)
  X_B_batch = shuffle_data(X_train_rot, batch_size)

  _, Disc_loss_epoch = sess.run([Disc_Optimize, Disc_Loss], feed_dict = {X_A:X_A_batch, X_B:X_B_batch})
  _, Gen_loss_epoch = sess.run([Gen_Optimize, Gen_Loss], feed_dict = {X_A:X_A_batch, X_B:X_B_batch})

  if epoch % 2000 == 0:
    print("steps: {0}, Disc loss; {1}, Gen Loss: {2}".format(epoch, Disc_loss_epoch, Gen_loss_epoch))


steps: 0, Disc loss; 0.5391185283660889, Gen Loss: 8.77369213104248
steps: 2000, Disc loss; 0.5032532811164856, Gen Loss: 8.50776481628418
steps: 4000, Disc loss; 0.45333611965179443, Gen Loss: 8.47620964050293
steps: 6000, Disc loss; 0.4041784405708313, Gen Loss: 8.444260597229004
steps: 8000, Disc loss; 0.33102962374687195, Gen Loss: 8.41366958618164
steps: 10000, Disc loss; 0.315215140581131, Gen Loss: 8.163247108459473
steps: 12000, Disc loss; 0.305846244096756, Gen Loss: 8.362942695617676
steps: 14000, Disc loss; 0.25823986530303955, Gen Loss: 8.4434814453125
steps: 16000, Disc loss; 0.2616346478462219, Gen Loss: 8.125432014465332
steps: 18000, Disc loss; 0.2449943721294403, Gen Loss: 8.320176124572754
steps: 20000, Disc loss; 0.2664198577404022, Gen Loss: 7.9900803565979
steps: 22000, Disc loss; 0.2840080261230469, Gen Loss: 7.727390766143799
steps: 24000, Disc loss; 0.2941964268684387, Gen Loss: 7.8364949226379395
steps: 26000, Disc loss; 0.27589184045791626, Gen Loss: 7.7939343

In [0]:
# Testing

n = 6
canvas1 = np.empty((28*n, 28*n))
canvas2 = np.empty((28*n, 28*n))

for i in range(n):
  test_A = shuffle_data(X_train_real, batch_size)
  test_B = shuffle_data(X_train_rot, batch_size)

  # Generate A images from B
  out_A = sess.run(X_BA, feed_dict={X_B: test_B})

  # Generate B images from A
  out_B = sess.run(X_AB, feed_dict={X_A: test_A})
  
  for j in range(n):
    # Draw the Generated Digits
    canvas1[i*28: (i + 1) *28, j* 28:(j+1) *28] = out_A[j].reshape([28,28])

  for j in range(n):
    # Draw the Generated Digits
    canvas2[i*28: (i + 1) *28, j* 28:(j+1) *28] = out_B[j].reshape([28,28])

# One way of displaying
plt.figure(figsize=(n, n))
plt.imshow(canvas1, origin = "upper", cmap = "gray")
plt.show()

plt.figure(figsize = (n,n))
plt.imshow(canvas2, origin="upper", cmap="gray")
plt.show()
